# Download an archive
The script below makes a data folder with a subfolder for the dataset. Dataset option are "World", "German" or "Dutch", select these by altering the ``dataset`` variable. The script wil skip files that are already present on the system. If you want to redownload the files first remove the old files manually.  
If the download fails due to connection errors, rerun the script and will will notice that he file did not decompress and try again.  
Note: The html-file-information.csv can be very large and take several hours and fail. Consider putting the code below code in a loop if you want to run it over night, if the download fails it will try again and if successfull it will say it already has the desired file.

In [1]:
# Dataset option are "World", "German" or "Dutch"
Dataset = "World"

!mkdir data
!mkdir data/{Dataset}

import os.path

if Dataset == "German":
    if not os.path.exists("data/German/domain-frequency.csv"):
        print("domain-frequency.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-germanNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/DomainFrequencyExtraction/domain-frequency.csv.gz?access=VRQ4COI5RFEB6XTJZNQTBRLEZTTHJERL" --output data/German/domain-frequency.csv.gz
        print("domain-frequency.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/German/domain-graph.csv"):
        print("domain-graph.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-germanNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/DomainGraphExtraction/domain-graph.csv.gz?access=JKEPGQ6MUC72JQB23IXOC4KOLGJYDSMN" --output data/German/domain-graph.csv.gz
        print("domain-graph.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/German/css-file-information.csv"):
        print("css-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-germanNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/css-file-information.csv.gz?access=I2WP4REJA3NOBU3TCAAL3OIGJKNXM46R" --output data/German/css-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("css-file-information.csv already exists")

    if not os.path.exists("data/German/js-file-information.csv"):
        print("js-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-germanNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/js-file-information.csv.gz?access=M3QSMFPLEHPZPWZIFSMZ6CT2OO7WYQ4M" --output data/German/js-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("js-file-information.csv already exists")
        
    if not os.path.exists("data/German/html-file-information.csv"):
        print("html-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-germanNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/html-file-information.csv.gz?access=E3BUHKL5P4Q3TD4LGXZV2AOMERYQ3GWL" --output data/German/html-file-information.csv.gz
        print("html-file-information.csv has been downloaded")
    else:
        print("html-file-information.csv already exists")
        
if Dataset == "World":
    if not os.path.exists("data/World/domain-frequency.csv"):
        print("domain-frequency.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-worldNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/DomainFrequencyExtraction/domain-frequency.csv.gz?access=SMSQY3G6IGKGRWVLGCWMA7DMCHBCKQ4K" --output data/World/domain-frequency.csv.gz
        print("domain-frequency.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/World/domain-graph.csv"):
        print("domain-graph.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-worldNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/DomainGraphExtraction/domain-graph.csv.gz?sample=true&access=BZTA7LW5LUNMPGKWQMLKAUAXBV2E2AEC" --output data/World/domain-graph.csv.gz
        print("domain-graph.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/World/css-file-information.csv"):
        print("css-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-worldNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/css-file-information.csv.gz?access=UZ4PWVRXXWPF53BHM7TTYNK24P7YAIXQ" --output data/World/css-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("css-file-information.csv already exists")

    if not os.path.exists("data/World/js-file-information.csv"):
        print("js-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-worldNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/js-file-information.csv.gz?access=54X7IV7QOOAJWGHKPRGDI7HIR6W6GKPI" --output data/World/js-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("js-file-information.csv already exists")
    
    if not os.path.exists("data/World/html-file-information.csv"):
        print("html-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-worldNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/html-file-information.csv.gz?access=JQEDT3PRXOS6OXZ7LA5EC55HQJVUASDX" --output data/World/html-file-information.csv.gz
        print("html-file-information.csv has been downloaded")
    else:
        print("html-file-information.csv already exists")

if Dataset == "Dutch":
    if not os.path.exists("data/Dutch/domain-frequency.csv"):
        print("domain-frequency.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-dutchNews_1st-in-month_1-hop_unique-EXTRACTION-20210916172606/DomainFrequencyExtraction/domain-frequency.csv.gz?access=QUQTRHVNIDKXN62S4D62XFHCFI7VDSH7" --output data/Dutch/domain-frequency.csv.gz
        print("domain-frequency.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/Dutch/domain-graph.csv"):
        print("domain-graph.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-dutchNews_1st-in-month_1-hop_unique-EXTRACTION-20210916172606/DomainGraphExtraction/domain-graph.csv.gz?access=JUNESOMDTBYDNCCGVJI5MJHW45KHAIJZ" --output data/Dutch/domain-graph.csv.gz
        print("domain-graph.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/Dutch/css-file-information.csv"):
        print("css-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-dutchNews_1st-in-month_1-hop_unique-EXTRACTION-20210916172606/TextFilesInformationExtraction/css-file-information.csv.gz?access=IGBKSGXJBL3L4IJ2LXS5NPJTOY77JCT5" --output data/Dutch/css-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("css-file-information.csv already exists")

    if not os.path.exists("data/Dutch/js-file-information.csv"):
        print("js-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-dutchNews_1st-in-month_1-hop_unique-EXTRACTION-20210916172606/TextFilesInformationExtraction/js-file-information.csv.gz?access=RPZKGWUTWJBIRKOKKHGECB7W4OHUATMO" --output data/Dutch/js-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("js-file-information.csv already exists")
    
    if not os.path.exists("data/Dutch/html-file-information.csv"):
        print("html-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-dutchNews_1st-in-month_1-hop_unique-EXTRACTION-20210916172606/TextFilesInformationExtraction/html-file-information.csv.gz?access=52ZOSNQQMFWKW42WMQPNMGKYLEVUQHUC" --output data/Dutch/html-file-information.csv.gz
        print("html-file-information.csv has been downloaded")
    else:
        print("html-file-information.csv already exists")

unzip = "find data/" +Dataset +" -name '*.gz' -exec gunzip {} \;"
!{unzip}

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/World’: File exists
domain-frequency.csv already exists
domain-frequency.csv already exists
css-file-information.csv already exists
js-file-information.csv already exists
html-file-information.csv already exists


# General helper functions

## Apply patch to fix pandas memory leak issue

The code bellow comes from https://github.com/pandas-dev/pandas/issues/2659#issuecomment-415177442  
It solves a memory leak issue caused by pandas using a library that does not release memory correctly causing memory to fill up over time and eventually crash if RAM is exceded.

This may not be a problem for smaller datasets

In [2]:
# monkeypatches.py

# Solving memory leak problem in pandas
# https://github.com/pandas-dev/pandas/issues/2659#issuecomment-12021083
import sys
import pandas as pd
from ctypes import cdll, CDLL
try:
    cdll.LoadLibrary("libc.so.6")
    libc = CDLL("libc.so.6")
    libc.malloc_trim(0)
except (OSError, AttributeError):
    libc = None

__old_del = getattr(pd.DataFrame, '__del__', None)

def __new_del(self):
    if __old_del:
        __old_del(self)
    libc.malloc_trim(0)

if libc:
    print('Applying monkeypatch for pd.DataFrame.__del__', file=sys.stderr)
    pd.DataFrame.__del__ = __new_del
else:
    print('Skipping monkeypatch for pd.DataFrame.__del__: libc or malloc_trim() not found', file=sys.stderr)

Applying monkeypatch for pd.DataFrame.__del__


## Convert a full url to a domain

In [3]:
from urllib import parse

def to_domain(full_url):
    netloc = parse.urlparse(full_url).netloc
    if netloc.startswith('www.'):
        netloc = netloc[4:]
    return netloc

## Filter dataframe with regex

The **chunksize** variable determines how much memory is used. A larger chunksize allows for faster computation, but will crash when RAM is full. If the computation crashes try restarting the kernel and/or a lower chunksize. A chunksize of 1000 should be fine for most machines. On our machine memory is not an issue, but still it is something to consider when running the same code on different machines.

In [4]:
import gc # For freeing up RAM
#import pandas as pd

def filter_df_content_regex(file, regex, chunksize):
    list_regex = [] # Appending to list before conferting to dataframe because it is computationally cheaper
    
    # Loops through the CSV in chunks and saves those that match the regex in the content column 
    for chunk in pd.read_csv(file, iterator=True, chunksize=chunksize):
        # Add all rows with the regex in the content column to list_regex
        list_regex.append(chunk[chunk.content.str.contains(regex, regex= True, na=False)])
        # chunk_offset = chunk_offset + chunksize
        del chunk
        gc.collect()
    filtered_df = pd.concat(list_regex, ignore_index=True) # Convert the list to a pandas dataframe

    return filtered_df

## Filter dataframe by date

In [5]:
def filter_df_date(file, begin_date, end_date , chunksize):
    list = [] # Appending to list before conferting to dataframe because it is computationally cheaper
    
    # Loops through the CSV in chunks and saves those that match the given timeframe
    for chunk in pd.read_csv(file, iterator=True, chunksize=chunksize):
        # Add all rows that match the given time frame
        list.append(chunk[(chunk['crawl_date'] >= begin_date) & (chunk['crawl_date'] <= end_date)])
        # chunk_offset = chunk_offset + chunksize
        del chunk
        gc.collect()
    filtered_df = pd.concat(list, ignore_index=True) # Convert the list to a pandas dataframe

    return filtered_df

## Add an Internet archive Wayback Machine link to a dataframe

Use the ``crawl_date`` and ``url`` columns to estimate the Internet Archive link. The CSV only provides the date of the crawl, not the time, so the resulting link may be for the wrong capure if more than one capture is available on that date. 

In [6]:
def add_ia_url(df):
    IA_url = []
    for index, row in df.iterrows():
        url = "https://web.archive.org/web/" + str(row['crawl_date']) + "/" + row["url"]
        IA_url.append(url)
    df['IA_url'] = IA_url

## Display a specific comment

In [7]:
def display_comment(dataframe, index):
    print(dataframe['comments'][index])

## Intersection of two dataframes

returns a dataframe with the intersection (overlap) of two dataframes

In [8]:
def dataframe_intersection(df1,df2):
    return pd.concat([df1,df2])[pd.concat([df1,df2]).duplicated(keep=False)].reset_index()

## Dataframe difference

returns a dataframe with all entries in each dataframe but not those in both

In [9]:
def dataframe_difference(df1,df2):
    return pd.concat([df1,df2]).drop_duplicates(keep=False, ignore_index=True)

    #pd.concat([df2, df1, df1]).drop_duplicates(keep=False) # would show what is in dataframe 2 that is not in dataframe 1 #TODO

## Restrict domain list

The code below filters out unwanted domains from the dataset an creates a new CSV file.  
The ``domains_in_scope`` input variable is a list of the desired domains

In [10]:
import gc # For freeing up RAM
import pandas as pd
import csv

def filter_df_domains(input_file, output_file, domains_in_scope, chunksize):
    output = open(output_file, "w")
    output.close()
    pd.read_csv(input_file, header=None, nrows=1).to_csv(output_file, mode='a', header=False)
    
    # Loops through the CSV in chunks and saves those with URLs thst match those in the domains_in_scope 
    for chunk in pd.read_csv(input_file, iterator=True, chunksize=chunksize):
        chunk=chunk[chunk.url.apply(to_domain).isin(domains_in_scope)]
        chunk.to_csv(output_file, mode='a', header=False)
        del chunk
        gc.collect()


#domains_in_scope = ["ksta.de"]
#filter_df_domains("data/German/html-file-information.csv", "data/German/ksta.csv", domains_in_scope, 10000)

# Extract wepages holding disqus.com/embed.js in their HTML

The code bellow looks through a dataset for any mention of disqus.js, an indicator of the presence of the disqus commenting system. The input for ```filter_df_content_regex``` is the desired dataset file, the regular expression and the chunksize.

In [11]:
# Options are "data/German/html-file-information.csv" "data/World/html-file-information.csv" and "data/Dutch/html-file-information.csv"
# or any other dataset on the system.
disqus_df = filter_df_content_regex("data/World/html-file-information.csv", "(?i)disqus\.com\/embed\.js", 10000)

Count the pages with disqus.js in their HTML

In [12]:
len(disqus_df)

6840

Note that this finds more hits than grep. It seems the csv file is too large for grep. the same query in awk does give the expected number of results

```grep -i -n 'disqus\.js' C/Users/rjans/Desktop/ARCH/Data/disqus.csv```  
vs  
```awk -e '/disqus\.js/ {print $0}' C/Users/rjans/Desktop/ARCH/Data/html-file-information-German.csv```

Show the unique domains

In [13]:
print(disqus_df['url'].apply(to_domain).unique())

['aljazeera.com' 'blogs.aljazeera.com' 'men.24.com' 'blogs.abcnews.com'
 'bloomberg.com' 'popwatch.ew.com' 'cnn.com' 'abcnews.go.com'
 'liveshots.blogs.foxnews.com' 'magazine.foxnews.com'
 'gretawire.foxnewsinsider.com' 'csmonitor.com' 'photos.essence.com'
 'myfoxchicago.com' 'myfoxla.com' 'myfoxmemphis.com' 'myfoxorlando.com'
 'mercurynews.com' 'spokesman.com' 'sfist.com' 'blogs.sfweekly.com'
 'tpmmuckraker.talkingpointsmemo.com' 'topgear.com' 'mirror.co.uk'
 'fashion.telegraph.co.uk' 'mweb.co.za' 'sagoodnews.co.za'
 'independent.co.uk' 'telegraph.co.uk' 'rawstory.com' 'pcmag.com'
 'politico.com' 'voices.news24.com' 'newser.com' 'newsmax.com'
 'english.aljazeera.net' 'lazygamer.net' 'washingtontimes.com'
 'communities.washingtontimes.com' 'propublica.org' 'scpr.org']


Save the results in a seperate CSV (only needs to be done once)

In [14]:
#disqus_df.to_csv("disqus.csv")

Convert **content** column to single line.

In [15]:
for line in disqus_df.content:
    disqus_df.content = disqus_df.content.replace(line, " ".join(line.splitlines()))

Save as single line content in a seperate CSV (only needs to be done once)

In [16]:
#disqus_df.to_csv("disqus_single_line.csv")

# For the found pages extract the comment structure

The code below requires beautifulsoup4. use the commented out import and pip command to install the library in the right place if you do not already beautifulsoup4

In [17]:
#import sys
#!{sys.executable} -m pip install beautifulsoup4
from bs4 import BeautifulSoup

Extract the disqus comments from the div tag with ```id="disqus_thread"```

In [18]:
comments = []
for line in disqus_df.content:
    soup = BeautifulSoup(line, 'html.parser')
    try:
        comments.append(soup.find('div', attrs={'id': 'disqus_thread'}).prettify()) # .prettify() makes the HTML of the comments more human readable
    except AttributeError:
        comments.append("")
        
disqus_df['comments'] = comments

In [19]:
#pd.set_option('display.max_colwidth', 500) 
disqus_df[['crawl_date','url','comments']]

,crawl_date,url,comments
0,20140502,http://www.aljazeera.com/news/middleeast/2014/...,"<div id=""disqus_thread"">\n</div>\n"
1,20140501,http://www.aljazeera.com/news/middleeast/2014/...,"<div id=""disqus_thread"">\n</div>\n"
2,20140502,http://www.aljazeera.com/news/middleeast/2014/...,"<div id=""disqus_thread"">\n</div>\n"
3,20140531,http://www.aljazeera.com/news/middleeast/2014/...,"<div id=""disqus_thread"">\n</div>\n"
4,20150102,http://www.aljazeera.com/news/middleeast/2014/...,"<div id=""disqus_thread"">\n</div>\n"
...,...,...,...
6835,20120105,http://www.independent.co.uk/environment/natur...,"<div id=""disqus_thread"">\n</div>\n"
6836,20110301,http://www.independent.co.uk/environment/scien...,"<div id=""disqus_thread"">\n <div class=""clearfi..."
6837,20120105,http://www.independent.co.uk/environment/the-1...,"<div id=""disqus_thread"">\n</div>\n"
6838,20130502,http://www.independent.co.uk/extras/indybest/o...,"<div id=""disqus_thread"">\n</div>\n"


# Add an internet archive URL to the dataframe

In [20]:
add_ia_url(disqus_df)

Filter out comment fields that are so short that they do net contain comments (based on the length).  
32 characters is currently the shortest 

In [21]:
#pd.set_option('display.max_colwidth', 50)
disqus_df[disqus_df['comments'].apply(lambda x: len(x)>32)][['comments','IA_url']]

,comments,IA_url
235,"<div class=""loading"" id=""disqus_thread"">\n</di...",https://web.archive.org/web/20150522/http://bl...
240,"<div class=""loading"" id=""disqus_thread"">\n</di...",https://web.archive.org/web/20150511/http://bl...
248,"<div id=""disqus_thread"">\n <script>\n var dis...",https://web.archive.org/web/20150127/http://ww...
249,"<div id=""disqus_thread"">\n <script>\n var dis...",https://web.archive.org/web/20140301/http://ww...
734,"<div class=""loading"" id=""disqus_thread"">\n</di...",https://web.archive.org/web/20150505/http://ab...
...,...,...
5529,"<div id=""disqus_thread"">\n <div class=""clearfi...",https://web.archive.org/web/20110412/http://ww...
5570,"<div class="""" id=""disqus_thread"">\n</div>\n",https://web.archive.org/web/20130304/http://ww...
6825,"<div id=""disqus_thread"">\n <div class=""clearfi...",https://web.archive.org/web/20110408/http://ww...
6828,"<div id=""disqus_thread"">\n <div class=""clearfi...",https://web.archive.org/web/20110927/http://ww...


Display a specific comment

In [22]:
display_comment(disqus_df, 0)

<div id="disqus_thread">
</div>



# Comparing dataframes

Creates a second dataframe with a different regex query 

In [23]:
disqus_df_2 = filter_df_content_regex("data/World/html-file-information.csv", "(?i)disqus\.js", 10000)

Show the overlap of two dataframes

In [24]:
dataframe_intersection(disqus_df[['crawl_date','url']],disqus_df_2[['crawl_date','url']])

,index,crawl_date,url
0,224,20180104,https://men.24.com/watch-five-crazy-sport-fans...
1,225,20180104,https://men.24.com/watch-five-crazy-sport-fans...
2,680,20140401,http://popwatch.ew.com/2014/03/31/how-i-met-yo...
3,681,20140401,http://popwatch.ew.com/2014/03/31/how-i-met-yo...
4,706,20100318,http://www.cnn.com/2009/SHOWBIZ/Music/12/31/ch...
...,...,...,...
207,1527,20170301,http://www.washingtontimes.com/news/2017/feb/2...
208,1530,20170301,http://www.washingtontimes.com/news/2017/feb/2...
209,1531,20170301,http://www.washingtontimes.com/news/2017/feb/2...
210,1913,20180201,https://www.washingtontimes.com/news/2018/jan/...


Show the difference between the two datasets. So only shows results that are in one dataset but not in both.

In [25]:
dataframe_difference(disqus_df[['crawl_date','url']],disqus_df_2[['crawl_date','url']])

,crawl_date,url
0,20140502,http://www.aljazeera.com/news/middleeast/2014/...
1,20140501,http://www.aljazeera.com/news/middleeast/2014/...
2,20140502,http://www.aljazeera.com/news/middleeast/2014/...
3,20140531,http://www.aljazeera.com/news/middleeast/2014/...
4,20150102,http://www.aljazeera.com/news/middleeast/2014/...
...,...,...
8539,20180201,https://www.washingtontimes.com/news/2018/jan/...
8540,20180201,https://www.washingtontimes.com/news/2018/jan/...
8541,20180201,https://www.washingtontimes.com/news/2018/jan/...
8542,20180201,https://www.washingtontimes.com/news/2018/jan/...
